## Fully Connected layer

In [3]:
%matplotlib inline
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
run_time = 4*t_max
dims = (1,10,10,2)
n_outputs = 100
weight_e = 500
weight_acc = 128
weight_args = {'weight_e':weight_e, 'weight_acc':weight_acc}

np.random.seed(seed=47)
weights = (np.random.rand(n_outputs,np.product(dims)//2) - 0.5) / 5
biases = (np.random.rand(n_outputs) - 0.5) / 2

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims, **weight_args),
    layers.FullyConnected(weights=weights, biases=None, **weight_args),
    layers.MonitorLayer(**weight_args),
    ])

values = np.random.rand(np.product(dims)//2)
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)

quantized_values = (values*t_max).round()/t_max
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(nn.Linear(in_features=np.product(dims[1:3]), out_features=n_outputs), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = torch.nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(quantized_values)).detach().numpy()

In [4]:
loihi_model.n_connections()

30800

In [ ]:
loihi_model(inputs, t_max)

--Return--
None
> /homes/glenz/STICK_on_loihi/stick/network.py(49)build_loihi_layered_model()
     48 
---> 49         ipdb.set_trace()
     50 



ipdb>  self.model.blocks


[]


ipdb>  self.model.prev_layer.blocks[0].neurons


[fc:l1-n  0-calc, fc:l1-n  0-sync, fc:l1-n  0-first, fc:l1-n  0-second, fc:l1-n  0-output]


ipdb>  self.model.prev_layer.blocks[0].neurons[0].loihi_type
ipdb>  self.model.neurons


[fc:l1-n  0-output-monitor, fc:l1-n  1-output-monitor, fc:l1-n  2-output-monitor, fc:l1-n  3-output-monitor, fc:l1-n  4-output-monitor, fc:l1-n  5-output-monitor, fc:l1-n  6-output-monitor, fc:l1-n  7-output-monitor, fc:l1-n  8-output-monitor, fc:l1-n  9-output-monitor, fc:l1-n 10-output-monitor, fc:l1-n 11-output-monitor, fc:l1-n 12-output-monitor, fc:l1-n 13-output-monitor, fc:l1-n 14-output-monitor, fc:l1-n 15-output-monitor, fc:l1-n 16-output-monitor, fc:l1-n 17-output-monitor, fc:l1-n 18-output-monitor, fc:l1-n 19-output-monitor, fc:l1-n 20-output-monitor, fc:l1-n 21-output-monitor, fc:l1-n 22-output-monitor, fc:l1-n 23-output-monitor, fc:l1-n 24-output-monitor, fc:l1-n 25-output-monitor, fc:l1-n 26-output-monitor, fc:l1-n 27-output-monitor, fc:l1-n 28-output-monitor, fc:l1-n 29-output-monitor, fc:l1-n 30-output-monitor, fc:l1-n 31-output-monitor, fc:l1-n 32-output-monitor, fc:l1-n 33-output-monitor, fc:l1-n 34-output-monitor, fc:l1-n 35-output-monitor, fc:l1-n 36-output-monitor, 

ipdb>  self.model.neurons[0].loihi_type


0


In [ ]:
output_values, spike_times = l2.run_on_loihi(run_time, t_max=t_max, input_spike_list=inputs, probe_selection=["calc", "sync"], 
                                             partition="nahuku32", num_chips=4, plot=False)
output_combinations = list(zip([value[0] for (key, value) in sorted(output_values.items())], model_output))
output_combinations

## Convolutional Layer

In [ ]:
import quartz
import numpy as np
import torch
import torch.nn as nn
t_max = 2**8
run_time = 8*t_max
input_dims=   (6, 5,  5, 2)
weight_dims = (32,6,5,5)
weight_e = 500
weight_acc = 128
kernel_size = weight_dims[2:]

l0 = quartz.layers.InputLayer(dims=input_dims, monitor=False, weight_e=weight_e, weight_acc=weight_acc)
weights = (np.random.rand(*weight_dims)-0.5) / 5
biases = (np.random.rand(weight_dims[0])-0.5) / 2
l1 = quartz.layers.Conv2D(prev_layer=l0, weights=weights, biases=biases, split_output=False,\
                         monitor=False, weight_e=weight_e, weight_acc=weight_acc)
l2 = quartz.layers.MonitorLayer(prev_layer=l1, weight_e=weight_e, weight_acc=weight_acc)

values = np.random.rand(np.product(input_dims)//2) / 2
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).round()/t_max
quantized_values = quantized_values.reshape(*input_dims[:3])
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=kernel_size), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = torch.nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()

In [ ]:
output_values, spike_times = l2.run_on_loihi(run_time, t_max=t_max, input_spike_list=inputs, probe_selection=["sync", "calc"], plot=False)
output_combinations = list(zip([value[0] for (key, value) in sorted(output_values.items())], model_output.flatten()))
output_combinations

## Max Pooling layer

In [ ]:
import quartz
import numpy as np
t_max = 2**8
run_time = 8*t_max
input_dims = (1,10,10,2)
kernel_size = [2,2]

l0 = quartz.layers.InputLayer(dims=input_dims, monitor=False)
l1 = quartz.layers.MaxPool2D(prev_layer=l0, kernel_size=kernel_size, split_output=True, monitor=False)
l2 = quartz.layers.MonitorLayer(prev_layer=l1)

np.random.seed(seed=45)
values = np.random.rand(np.product(dims)//2)
inputs = quartz.utils.decode_values_into_spike_input(values, t_max)
quantized_values = (values*t_max).astype(int)/t_max

model = nn.Sequential(nn.MaxPool2d(kernel_size=kernel_size, stride=stride), nn.ReLU())
model_output = model(torch.tensor(quantized_values.reshape(1, *input_dims[:3]))).squeeze().detach().numpy()

In [ ]:
output_values, spike_times = l2.run_on_loihi(run_time, t_max=t_max, input_spike_list=inputs, plot=False)
output_values